<a href="https://colab.research.google.com/github/felixyustian/langgraph_agent_test/blob/main/AI_Agents_Revamped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Agents with LangGraph Part 1

In [ ]:
c h0%pip install -qU langgraph langchain-google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 17.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


## Exercise 1: SQL Agents

SQL agents adalah AI Agent yang dapat mengambil data dari table SQL dan menganalisis sesuai dengan pertanyaan dari user.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os

from google.colab import userdata
GEMINI = userdata.get('GEMINI')
os.environ["GOOGLE_API_KEY"] = GEMINI

### Langkah 1: Define beberapa table SQL

In [ ]:
%load_ext sql
%sql sqlite:///sample.db

In [ ]:
%%sql
-- Create the 'products' table
CREATE TABLE IF NOT EXISTS products (
  	product_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	product_name VARCHAR(255) NOT NULL,
  	price DECIMAL(10, 2) NOT NULL
  );

-- Create the 'staff' table
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

-- Create the 'orders' table
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );

-- Insert data into the 'products' table
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

-- Insert data into the 'staff' table
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams');

-- Insert data into the 'orders' table
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
Done.
Done.
3 rows affected.
3 rows affected.
3 rows affected.


[]

In [ ]:
import sqlite3

db_file = "sample.db"

### Langkah 2: Define Tools
Tools yang akan digunakan pada SQL agent adalah `list_tables`, `describe_tables`, dan `execute_query` sehingga AI dapat memahami metadata table terlebih dahulu sebelum mulai menjalankan query

In [ ]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables')

    # Create a new connection for thread safety
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        # Fetch the table names.
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

        tables = cursor.fetchall()
        return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables


['products', 'sqlite_sequence', 'staff', 'orders']

In [ ]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(' - DB CALL: describe_table')

    # Create a new connection for thread safety
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        cursor.execute(f"PRAGMA table_info({table_name});")

        schema = cursor.fetchall()
        # [column index, column name, column type, ...]
        return [(col[1], col[2]) for col in schema]


describe_table("products")

 - DB CALL: describe_table


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

In [ ]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute a SELECT statement, returning the results."""
    print(' - DB CALL: execute_query')

    # Create a new connection for thread safety
    with sqlite3.connect(db_file) as conn:
        cursor = conn.cursor()

        cursor.execute(sql)
        return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

### Langkah 3: Bangun chatbot dengan create_react_agent

In [ ]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database for a computer
store. You will take the users questions and turn them into SQL queries using the tools
available. Once you have the information you need, you will answer the user's question using
the data returned. Use list_tables to see what tables are present, describe_table to understand
the schema, and execute_query to issue an SQL SELECT query."""

# Import the necessary modules from langgraph
from langgraph.prebuilt import create_react_agent

# Initialize the chat model with specific parameters
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)

# Create a ReAct agent using the model, tools, and prompt
agent = create_react_agent(
    model=model,
    tools=db_tools,
    prompt=instruction
)

perhatikan respons chatbot di bawah, LLM memanggil tool list_table -> describe_table -> execute_query untuk menganalisis jawaban dan memberikan jawaban yang tepat.

In [ ]:
# Run the agent
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Name all employee who works in the company"}]}
)

print(response["messages"][-1].content)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
The employees of the company are Alice Smith, Bob Johnson, and Charlie Williams.


In [ ]:
# Run the agent with a different query
response = agent.invoke(
    {"messages": [{"role": "user", "content": "Produk apa yang harganya paling mahal"}]}
)

print(response["messages"][-1].content)

 - DB CALL: list_tables
 - DB CALL: describe_table
 - DB CALL: execute_query
Produk termahal adalah Laptop dengan harga 799.99.


# AI Agents with LangGraph Part 2

## Langkah 1: Define State dan Prompt

In [ ]:
system_instruction = """
You are a BaristaBot, an interactive cafe ordering system.
A human will talk to you about the available products you have and you will answer any questions about menu items
(and only about menu items - no off-topic discussion, but you can chat about the products and their history).
The customer will place an order for 1 or more items from the menu, which you will structure and send to the ordering system after confirming the order with the human.

Add items to the customer's order with add_to_order, and reset the order with clear_order.
To see the contents of the order so far, call get_order (this is shown to you, not the user)
Always confirm_order with the user (double-check) before calling place_order.
Calling confirm_order will display the order items to the user and returns their response to seeing the list.
Their response may contain modifications. Always verify and respond with drink and modifier names from the MENU before adding them to the order.
If you are unsure a drink or modifier matches those on the MENU, ask a question to clarify or redirect.
You only have the modifiers listed on the menu.
Once the customer has finished ordering items, Call confirm_order to ensure it is correct then make any necessary updates and then call place_order.
Once place_order has returned, thank the user and say goodbye!
"""

## Langkah 2: Define Flow Chatbot dengan create_react_agent

In [ ]:
# In-memory database for the customer's order
customer_order = []

def get_menu() -> str:
    """Provide the latest up-to-date menu."""
    # Note that this is just hard-coded text, but you could connect this to a live stock
    # database, or you could use Gemini's multi-modal capabilities and take live photos of
    # your cafe's chalk menu or the products on the counter and assmble them into an input.

    return """
    MENU:
    Coffee Drinks:
    Espresso
    Americano
    Cold Brew

    Coffee Drinks with Milk:
    Latte
    Cappuccino
    Cortado
    Macchiato
    Mocha
    Flat White

    Tea Drinks:
    English Breakfast Tea
    Green Tea
    Earl Grey

    Tea Drinks with Milk:
    Chai Latte
    Matcha Latte
    London Fog

    Other Drinks:
    Steamer
    Hot Chocolate

    Modifiers:
    Milk options: Whole, 2%, Oat, Almond, 2% Lactose Free; Default option: whole
    Espresso shots: Single, Double, Triple, Quadruple; default: Double
    Caffeine: Decaf, Regular; default: Regular
    Hot-Iced: Hot, Iced; Default: Hot
    Sweeteners (option to add one or more): vanilla sweetener, hazelnut sweetener, caramel sauce, chocolate sauce, sugar free vanilla sweetener
    Special requests: any reasonable modification that does not involve items not on the menu, for example: 'extra hot', 'one pump', 'half caff', 'extra foam', etc.

    "dirty" means add a shot of espresso to a drink that doesn't usually have it, like "Dirty Chai Latte".
    "Regular milk" is the same as 'whole milk'.
    "Sweetened" means add some regular sugar, not a sweetener.

    Soy milk has run out of stock today, so soy is not available.
    """

def add_to_order(item: str) -> str:
    """Add an item to the customer's order."""
    global customer_order
    customer_order.append(item)
    print(f"Adding '{item}' to order. Current order: {customer_order}")
    return f"I've added '{item}' to your order."

def clear_order() -> str:
    """Clear all items from the customer's order."""
    global customer_order
    customer_order.clear()
    print("Clearing order.")
    return "Your order has been cleared."

def get_order() -> list[str]:
    """Get the current items in the customer's order."""
    global customer_order
    print(f"Getting order. Current order: {customer_order}")
    return customer_order

def confirm_order() -> str:
    """Confirm the order with the customer."""
    global customer_order
    print(f"Confirming order. Current order: {customer_order}")
    if not customer_order:
        return "Your order is currently empty. What can I get for you?"

    order_string = ", ".join(customer_order)
    return f"Your order contains: {order_string}. Is this correct?"

def place_order() -> str:
    """Place the final order."""
    global customer_order
    print(f"Placing order. Final order: {customer_order}")
    if not customer_order:
        return "There's nothing in your order to place."

    final_order_summary = ", ".join(customer_order)
    # Clear the order after placing it, ready for the next customer
    customer_order.clear()
    return f"Your order for '{final_order_summary}' has been placed! It will be ready shortly."

# Collect all tools
barista_tools = [get_menu, add_to_order, clear_order, get_order, confirm_order, place_order]

# Import necessary modules
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

# Initialize the chat model
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.2
)

# Create a checkpointer for memory
checkpointer = InMemorySaver()

# Create the ReAct agent
barista_agent = create_react_agent(
    model=model,
    tools=barista_tools,
    prompt=system_instruction,
    checkpointer=checkpointer
)

### Interaksi dengan Barista Bot

In [ ]:
# Function to interact with the barista bot
def chat_with_barista():
    thread_id = "1"  # Unique identifier for this conversation
    config = {"configurable": {"thread_id": thread_id}}

    print("Welcome to BaristaBot! Type 'q' to quit.")

    # First response with welcome message
    response = barista_agent.invoke(
        {"messages": []},
        config=config
    )
    print("BaristaBot:", response["messages"][-1]["content"])

    while True:
        user_input = input("You: ")
        if user_input.lower() in ['q', 'quit', 'exit']:
            print("BaristaBot: Thank you for visiting! Have a great day!")
            break

        response = barista_agent.invoke(
            {"messages": [{"role": "user", "content": user_input}]},
            config=config
        )

        print("BaristaBot:", response["messages"][-1]["content"])

In [ ]:
# Run the interactive chat
# Uncomment the line below to start chatting with the barista bot
# chat_with_barista()

### Contoh Penggunaan Langsung

In [ ]:
# Example of direct usage with specific queries
thread_id = "3"  # Unique identifier for this conversation
config = {"configurable": {"thread_id": thread_id}}

# User asks about menu
response = barista_agent.invoke(
    {"messages": [{"role": "user", "content": "What drinks do you have?"}]},
    config=config
)
print("User: What drinks do you have?")
print("BaristaBot:", response["messages"][-1].content)

# User orders a drink
response = barista_agent.invoke(
    {"messages": [{"role": "user", "content": "I'd like a green tea please"}]},
    config=config
)
print("User: I'd like a green tea please")
print("BaristaBot:", response["messages"][-1].content)

# User orders a drink
response = barista_agent.invoke(
    {"messages": [{"role": "user", "content": "what is my order?"}]},
    config=config
)
print("User: what is my order?")
print("BaristaBot:", response["messages"][-1].content)

User: What drinks do you have?
BaristaBot: We have a variety of coffee drinks, including Espresso, Americano, and Cold Brew. We also have coffee drinks with milk like Latte, Cappuccino, Cortado, Macchiato, Mocha, and Flat White.

If you prefer tea, we offer English Breakfast Tea, Green Tea, and Earl Grey. Our tea drinks with milk include Chai Latte, Matcha Latte, and London Fog.

Additionally, we have Steamer and Hot Chocolate.
Adding 'Green Tea' to order. Current order: ['Green Tea']
Getting order. Current order: ['Green Tea']
User: I'd like a green tea please
BaristaBot: Ok, I've added a Green Tea to your order. Anything else?
Confirming order. Current order: ['Green Tea']
User: what is my order?
BaristaBot: Your order contains: Green Tea. Is this correct?
